In [1]:
# GetOldTweet3 사용 준비
try:
    import GetOldTweets3 as got
except:
    !pip install GetOldTweets3
    import GetOldTweets3 as got

In [2]:
# BeautifulSoup4 사용 준비
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import BeautifulSoup

In [4]:
'''
data.Start_date = 시작 일자
data.End_date = 끝나는 일자
data.검색명 = 검색어 

'''


In [6]:
import requests
def get_bs_obj(url):
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    return bs_obj



def Tweet_C_date(data) :
    '''
    S_d = Start_date
    E_d = End_date
    Name = Search for Name or Object
    출처 = https://jeongwookie.github.io/2019/06/10/190610-twitter-data-crawling/
    
    '''
    S_d = data.Start_date
    E_d = data.End_date
    Name = data.검색명
    
    
    import datetime
    days_range = []

    start = datetime.datetime.strptime(S_d, "%Y-%m-%d")
    end = datetime.datetime.strptime(E_d, "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

    for date in date_generated:
        days_range.append(date.strftime("%Y-%m-%d"))

    print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
    print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))
    


    import time

    # 수집 기간 맞추기
    start_date = days_range[0]
    end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
                + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1

    # 트윗 수집 기준 정의
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(Name)\
                                               .setSince(start_date)\
                                               .setUntil(end_date)\
                                               .setMaxTweets(-1)

    # 수집 with GetOldTweet3
    print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
    start_time = time.time()

    tweet = got.manager.TweetManager.getTweets(tweetCriteria)

    print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
    print("=== Total num of tweets is {} ===".format(len(tweet)))
    '''return(tweet)'''
    
    from random import uniform
    from tqdm import tqdm_notebook
    import time
    # initialize
    tweet_list = []

    for index in tqdm_notebook(tweet):

        # 메타데이터 목록 
        username = index.username
        link = index.permalink 
        content = index.text
        tweet_date = index.date.strftime("%Y-%m-%d")
        tweet_time = index.date.strftime("%H:%M:%S")
        retweets = index.retweets
        favorites = index.favorites

        # === 유저 정보 수집 시작 ===
        '''try:
            personal_link = 'https://twitter.com/' + username
            bs_obj = get_bs_obj(personal_link)
            uls = bs_obj.find("ul", {"class": "ProfileNav-list"}).find_all("li")
            div = bs_obj.find("div", {"class": "ProfileHeaderCard-joinDate"}).find_all("span")[1]["title"]


            # 가입일, 전체 트윗 수, 팔로잉 수, 팔로워 수
            joined_date = div.split('-')[1].strip()
            num_tweets = uls[0].find("span", {"class": "ProfileNav-value"}).text.strip()
            num_following = uls[1].find("span", {"class": "ProfileNav-value"}).text.strip()
            num_follower = uls[2].find("span", {"class": "ProfileNav-value"}).text.strip()
        '''
        '''except AttributeError:
            print("=== Attribute error occurs at {} ===".format(link))
            print("link : {}".format(personal_link))   
            pass
        '''
        # 결과 합치기
        info_list = [tweet_date, tweet_time, username, content]#, link, retweets, favorites, 
                     #joined_date, num_tweets, num_following, num_follower]
        tweet_list.append(info_list)

        # 휴식 
        time.sleep(uniform(1,2))

    twitter_df = pd.DataFrame(tweet_list, 
                                  columns = ["date", "time", "user_name", "text"])#, "link", "retweet_counts", "favorite_counts",
                                            #"user_created", "user_tweets", "user_followings", "user_followers"])
    twitter_df.to_csv('./'+S_d+data.후보명+"Tweet.csv".format(days_range[0], days_range[-1]), index=False)
    print(data.후보명+"=== {} tweets are successfully saved ===".format(len(tweet_list)))



In [13]:
# 원하는 변수 골라서 저장하기
for x in range(data.shape[0]) :
    print(data.iloc[x])
    tweet = Tweet_C_date(data.iloc[x])
        

선거명           제 21대 국회의원선거
후보명                    이낙연
Start_date      2020-04-02
End_date        2020-04-14
Name: 0, dtype: object
=== 설정된 트윗 수집 기간은 2020-04-02 에서 2020-04-13 까지 입니다 ===
=== 총 12일 간의 데이터 수집 ===
=== Total num of tweets is 7690 ===


C:\Users\82105\Anaconda3\lib\site-packages\ipykernel\__main__.py:64: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



이낙연=== 7690 tweets are successfully saved ===
선거명           제 21대 국회의원선거
후보명                    황교안
Start_date      2020-04-02
End_date        2020-04-14
Name: 1, dtype: object
=== 설정된 트윗 수집 기간은 2020-04-02 에서 2020-04-13 까지 입니다 ===
=== 총 12일 간의 데이터 수집 ===
An error occured during an HTTP request: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>
Try to open in browser: https://twitter.com/search?q=%ED%99%A9%EA%B5%90%EC%95%88%20since%3A2020-04-02%20until%3A2020-04-14&src=typd


SystemExit: 

C:\Users\82105\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
